In [1]:
!pip install --upgrade --no-cache-dir transformers==4.45.2 accelerate datasets sentencepiece indic-transliteration -q
!pip install scikit-learn pandas tqdm matplotlib -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 120.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 352.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 224.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 335.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 332.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 361.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 116.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 232.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 172.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 166.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 201.0 MB/s eta 0:00:00a 0:00

# Imports, Config

In [3]:
import os, json, torch, random
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
DATA_DIR = Path("/kaggle/input/dataaaaaaa")
POS_FILE = DATA_DIR / "all_positive_8500.txt"
NEG_FILE = DATA_DIR / "all_negative_3307.txt"

# Training settings
BATCH_SIZE = 16
MAX_LEN = 128
EPOCHS_TEACHER = 3
EPOCHS_STUDENT = 4
LR_TEACHER = 2e-5
LR_STUDENT = 3e-5
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.1
KD_T = 3.0
KD_ALPHA = 0.5
GAMMA_HIDDEN = 1.0
GAMMA_ATT = 1.0

# --------------------------------------------------------------
# Load Bangla dataset
# --------------------------------------------------------------
def read_txt(p):
    with open(p, encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

pos_texts = read_txt(POS_FILE)
neg_texts = read_txt(NEG_FILE)

df = pd.DataFrame({
    "text": pos_texts + neg_texts,
    "label": [1]*len(pos_texts) + [0]*len(neg_texts)
})
df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=SEED)
val_df, test_df   = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=SEED)

print(f"✅ Dataset loaded: train={len(train_df)}, val={len(val_df)}, test={len(test_df)}")


✅ Dataset loaded: train=9445, val=1181, test=1181


# Train teacher (BanglaBERT) and save best

In [4]:
# ==============================================================
# 🧠 Fine-tune Teacher Model (XLM-R)
# ==============================================================

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn

MODEL_ID = "xlm-roberta-base"
NUM_LABELS = 2
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01

teacher_tok = AutoTokenizer.from_pretrained(MODEL_ID)
teacher = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=NUM_LABELS).to(DEVICE)

class TextDataset(Dataset):
    def __init__(self, df, tok, max_len):
        self.texts = df.text.tolist()
        self.labels = df.label.tolist()
        self.tok = tok
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = self.tok(self.texts[i], truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        return {"input_ids": enc["input_ids"].squeeze(), "attention_mask": enc["attention_mask"].squeeze(),
                "labels": torch.tensor(self.labels[i], dtype=torch.long)}

train_loader = DataLoader(TextDataset(train_df, teacher_tok, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(TextDataset(val_df, teacher_tok, MAX_LEN), batch_size=BATCH_SIZE)
test_loader  = DataLoader(TextDataset(test_df, teacher_tok, MAX_LEN), batch_size=BATCH_SIZE)

optimizer = AdamW(teacher.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
steps = len(train_loader)*EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, int(WARMUP_RATIO*steps), steps)
criterion = nn.CrossEntropyLoss()

SAVE_DIR = "/kaggle/working/fine_tuned_xlmr"
os.makedirs(SAVE_DIR, exist_ok=True)

best_val_f1 = 0.0
for ep in range(1, EPOCHS+1):
    teacher.train(); total_loss = 0
    for b in tqdm(train_loader, desc=f"Epoch {ep}/{EPOCHS}"):
        b = {k:v.to(DEVICE) for k,v in b.items()}
        out = teacher(**b)
        loss = out.loss
        loss.backward(); optimizer.step(); scheduler.step(); optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss/len(train_loader):.4f}")

    teacher.eval(); preds, gold = [], []
    with torch.no_grad():
        for b in val_loader:
            b = {k:v.to(DEVICE) for k,v in b.items()}
            out = teacher(**b)
            preds += out.logits.argmax(-1).cpu().tolist()
            gold  += b["labels"].cpu().tolist()
    acc = accuracy_score(gold, preds); f1 = f1_score(gold, preds, average="macro")
    print(f"Val Acc={acc:.4f} | F1={f1:.4f}")
    if f1>best_val_f1:
        best_val_f1=f1
        teacher.save_pretrained(SAVE_DIR)
        teacher_tok.save_pretrained(SAVE_DIR)
        print(f"✅ Saved best model to {SAVE_DIR}")

print(f"Best Val F1={best_val_f1:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

Train Loss: 0.3318
Val Acc=0.9526 | F1=0.9416
✅ Saved best model to /kaggle/working/fine_tuned_xlmr


Epoch 2/3:   0%|          | 0/591 [00:00<?, ?it/s]

Train Loss: 0.1241
Val Acc=0.9602 | F1=0.9506
✅ Saved best model to /kaggle/working/fine_tuned_xlmr


Epoch 3/3:   0%|          | 0/591 [00:00<?, ?it/s]

Train Loss: 0.0795
Val Acc=0.9636 | F1=0.9548
✅ Saved best model to /kaggle/working/fine_tuned_xlmr
Best Val F1=0.9548


# Transliteration KD Dataset

In [7]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

def transliterate_bn_text(text: str) -> str:
    try:
        return transliterate(text, sanscript.BENGALI, sanscript.ITRANS)
    except Exception:
        return text

STUDENT_MODEL_ID = "distilbert-base-multilingual-cased"
student_tok = AutoTokenizer.from_pretrained(STUDENT_MODEL_ID)

class KDDataset(Dataset):
    def __init__(self, df, teacher_tok, student_tok, max_len):
        self.texts = df.text.tolist()
        self.labels = df.label.tolist()
        self.teacher_tok = teacher_tok
        self.student_tok = student_tok
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        text_bn = self.texts[i]; text_en = transliterate_bn_text(text_bn)
        t_enc = self.teacher_tok(text_bn, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        s_enc = self.student_tok(text_en, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        return {"t_input_ids": t_enc["input_ids"].squeeze(0), "t_attention_mask": t_enc["attention_mask"].squeeze(0),
                "s_input_ids": s_enc["input_ids"].squeeze(0), "s_attention_mask": s_enc["attention_mask"].squeeze(0),
                "labels": torch.tensor(self.labels[i], dtype=torch.long)}

def pad_collate(batch, t_pad_id, s_pad_id):
    out = {}
    for k in batch[0].keys():
        if k=="labels":
            out[k]=torch.stack([b[k] for b in batch])
        elif k.startswith("t_"):
            pad_val=0 if "attention" in k else t_pad_id
            out[k]=nn.utils.rnn.pad_sequence([b[k] for b in batch],batch_first=True,padding_value=pad_val)
        elif k.startswith("s_"):
            pad_val=0 if "attention" in k else s_pad_id
            out[k]=nn.utils.rnn.pad_sequence([b[k] for b in batch],batch_first=True,padding_value=pad_val)
    return out

train_loader = DataLoader(KDDataset(train_df, teacher_tok, student_tok, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=lambda b: pad_collate(b, teacher_tok.pad_token_id, student_tok.pad_token_id))
val_loader   = DataLoader(KDDataset(val_df, teacher_tok, student_tok, MAX_LEN), batch_size=BATCH_SIZE,
                          collate_fn=lambda b: pad_collate(b, teacher_tok.pad_token_id, student_tok.pad_token_id))
test_loader  = DataLoader(KDDataset(test_df, teacher_tok, student_tok, MAX_LEN), batch_size=BATCH_SIZE,
                          collate_fn=lambda b: pad_collate(b, teacher_tok.pad_token_id, student_tok.pad_token_id))

print("✅ KD dataloaders ready.")


✅ KD dataloaders ready.


# Student model + Logit + Hidden + Attention KD loss

In [9]:
# ==============================================================
# 🎓 Knowledge Distillation Training (Logits + Hidden + Attention)
# ==============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from transformers import AdamW, get_linear_schedule_with_warmup
from pathlib import Path

# ==============================================================
# 📁 Define Working Directory
# ==============================================================
WORK_DIR = Path("/kaggle/working")
WORK_DIR.mkdir(parents=True, exist_ok=True)
print(f"📂 Working directory set to: {WORK_DIR}")

# ==============================================================
# 🧠 KD Loss Definition
# ==============================================================
class KDLossFull(nn.Module):
    def __init__(self, T=3.0, alpha=0.5, gamma_h=1.0, gamma_a=1.0):
        super().__init__()
        self.T = T
        self.alpha = alpha
        self.gamma_h = gamma_h
        self.gamma_a = gamma_a
        self.ce = nn.CrossEntropyLoss()
        self.kld = nn.KLDivLoss(reduction="batchmean")
        self.mse = nn.MSELoss()

    @staticmethod
    def map_layers(n_s, n_t):
        """Map student layers to teacher layers proportionally."""
        s_idx = list(range(1, n_s + 1))
        t_idx = torch.linspace(1, n_t, steps=n_s).round().long().tolist()
        return list(zip(s_idx, t_idx))

    def forward(self, s_pack, t_pack, labels):
        # ---------- Logits KD ----------
        logits_s, logits_t = s_pack["logits"], t_pack["logits"]
        hard = self.ce(logits_s, labels)
        soft = self.kld(F.log_softmax(logits_s / self.T, dim=-1),
                        F.softmax(logits_t / self.T, dim=-1)) * (self.T ** 2)
        total = (1 - self.alpha) * hard + self.alpha * soft

        # ---------- Hidden State KD ----------
        h_s, h_t = s_pack.get("hidden_states", []), t_pack.get("hidden_states", [])
        if h_s and h_t:
            pairs = self.map_layers(len(h_s) - 1, len(h_t) - 1)
            h_losses = []
            for i_s, i_t in pairs:
                hs, ht = h_s[i_s], h_t[i_t]
                seq = min(hs.size(1), ht.size(1))
                h_losses.append(self.mse(hs[:, :seq, :], ht[:, :seq, :]))
            if h_losses:
                total += self.gamma_h * torch.stack(h_losses).mean()

        # ---------- Attention Map KD (Safe) ----------
        a_s, a_t = s_pack.get("attentions", []), t_pack.get("attentions", [])
        if a_s and a_t:
            pairs = self.map_layers(len(a_s), len(a_t))
            a_losses = []
            for i_s, i_t in pairs:
                try:
                    as_ = a_s[i_s]
                    at_ = a_t[i_t]
                    if as_ is None or at_ is None or as_.dim() != 4 or at_.dim() != 4:
                        continue
                    seq = min(as_.size(-1), at_.size(-1))
                    a_losses.append(self.mse(as_[:, :, :seq, :seq], at_[:, :, :seq, :seq]))
                except Exception:
                    continue
            if a_losses:
                total += self.gamma_a * torch.stack(a_losses).mean()

        return total


# ==============================================================
# ⚙️ Setup Optimizer / Scheduler
# ==============================================================
LR_STUDENT = 3e-5
EPOCHS_STUDENT = 5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
PATIENCE = 2

criterion = KDLossFull(T=3.0, alpha=0.5, gamma_h=1.0, gamma_a=1.0)

# Freeze teacher
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

opt_s = AdamW(student.parameters(), lr=LR_STUDENT, weight_decay=WEIGHT_DECAY)
num_steps_s = EPOCHS_STUDENT * len(train_loader)
warm_steps_s = int(WARMUP_RATIO * num_steps_s)
sch_s = get_linear_schedule_with_warmup(opt_s, warm_steps_s, num_steps_s)

best_f1, wait = -1, 0


# ==============================================================
# 📏 Helper: Compute Metrics
# ==============================================================
def compute_metrics(preds, gold):
    return {
        "accuracy": accuracy_score(gold, preds),
        "f1_macro": f1_score(gold, preds, average="macro"),
        "f1_weighted": f1_score(gold, preds, average="weighted")
    }

@torch.no_grad()
def eval_student(loader):
    student.eval()
    preds, gold = [], []
    for b in loader:
        b = {k: v.to(DEVICE) for k, v in b.items()}
        out = student(input_ids=b["s_input_ids"], attention_mask=b["s_attention_mask"])
        preds += out["logits"].argmax(-1).cpu().tolist()
        gold  += b["labels"].cpu().tolist()
    return compute_metrics(np.array(preds), np.array(gold))


# ==============================================================
# 🚀 KD Training Loop
# ==============================================================
for ep in range(1, EPOCHS_STUDENT + 1):
    student.train()
    run_loss = 0.0

    for b in tqdm(train_loader, desc=f"[KD Epoch {ep}/{EPOCHS_STUDENT}]", leave=False):
        labels = b["labels"].to(DEVICE)

        # ---- Forward Student ----
        s_out = student(
            input_ids=b["s_input_ids"].to(DEVICE),
            attention_mask=b["s_attention_mask"].to(DEVICE),
            output_hidden_states=True,
            output_attentions=True
        )

        # ---- Forward Teacher ----
        with torch.no_grad():
            t_raw = teacher(
                input_ids=b["t_input_ids"].to(DEVICE),
                attention_mask=b["t_attention_mask"].to(DEVICE),
                output_hidden_states=True,
                output_attentions=True,
                return_dict=True
            )
            t_out = {
                "logits": t_raw.logits,
                "hidden_states": t_raw.hidden_states,
                "attentions": t_raw.attentions
            }

        # ---- Compute KD Loss ----
        loss = criterion(s_out, t_out, labels)
        loss.backward()

        nn.utils.clip_grad_norm_(student.parameters(), 1.0)
        opt_s.step(); sch_s.step(); opt_s.zero_grad()
        run_loss += loss.item()

    # ---- Validation ----
    val = eval_student(val_loader)
    print(f"[KD Epoch {ep}] loss={run_loss/len(train_loader):.4f} | "
          f"Val F1m={val['f1_macro']:.4f} Acc={val['accuracy']:.4f}")

    # ---- Early Stopping ----
    if val["f1_macro"] > best_f1:
        best_f1, wait = val["f1_macro"], 0
        torch.save(student.state_dict(), WORK_DIR / "student_best_kd.pt")
        print("✅ Saved best student checkpoint.")
    else:
        wait += 1
        if wait >= PATIENCE:
            print("⏸️ Early stopping triggered.")
            break

# ==============================================================
# 🧩 Reload Best Model
# ==============================================================
student.load_state_dict(torch.load(WORK_DIR / "student_best_kd.pt", map_location=DEVICE))
student.eval()
print("✅ Student KD training complete (logits + hidden + attention).")


📂 Working directory set to: /kaggle/working


NameError: name 'student' is not defined

In [14]:
# ==============================================================
# 📊 Evaluate Teacher vs Student (Performance + Alignment) + Save
# ==============================================================

from sklearn.metrics import accuracy_score, f1_score
from scipy.special import softmax
from scipy.spatial.distance import cosine
import numpy as np
import json
from pathlib import Path
import torch

teacher.eval(); student.eval()

# ---------- helpers ----------
@torch.no_grad()
def evaluate_model(model, loader, mode="student"):
    preds, gold, probs = [], [], []
    for b in loader:
        b = {k: v.to(DEVICE) for k, v in b.items()}
        if mode == "teacher":
            inp = {"input_ids": b["t_input_ids"], "attention_mask": b["t_attention_mask"]}
        else:
            inp = {"input_ids": b["s_input_ids"], "attention_mask": b["s_attention_mask"]}
        out = model(**inp)
        logits = out["logits"] if isinstance(out, dict) else out.logits
        p = logits.softmax(-1)
        preds += p.argmax(-1).cpu().tolist()
        gold  += b["labels"].cpu().tolist()
        probs += p[:, 1].cpu().tolist()
    return {
        "accuracy": accuracy_score(gold, preds),
        "f1_macro": f1_score(gold, preds, average="macro"),
        "f1_weighted": f1_score(gold, preds, average="weighted")
    }, np.array(probs), np.array(preds), np.array(gold)

@torch.no_grad()
def evaluate_alignment(teacher, student, loader):
    logits_cos, prob_corr = [], []
    t_preds_all, s_preds_all = [], []
    for b in loader:
        b = {k: v.to(DEVICE) for k, v in b.items()}
        t = teacher(input_ids=b["t_input_ids"], attention_mask=b["t_attention_mask"])
        s = student(input_ids=b["s_input_ids"], attention_mask=b["s_attention_mask"])
        t_logits = t.logits.detach().cpu().numpy()
        s_logits = s["logits"].detach().cpu().numpy()
        t_probs = softmax(t_logits, axis=-1)
        s_probs = softmax(s_logits, axis=-1)
        for tl, sl, tp, sp in zip(t_logits, s_logits, t_probs, s_probs):
            logits_cos.append(1 - cosine(tl, sl))
            prob_corr.append(np.corrcoef(tp, sp)[0, 1])
            t_preds_all.append(np.argmax(tp))
            s_preds_all.append(np.argmax(sp))
    return {
        "logit_cosine": float(np.nanmean(logits_cos)),
        "prob_corr": float(np.nanmean(prob_corr)),
        "pred_alignment": float((np.array(t_preds_all) == np.array(s_preds_all)).mean())
    }

# ---------- evaluate ----------
print("Evaluating Teacher on Test set ...")
teacher_test, t_probs, t_preds, gold = evaluate_model(teacher, test_loader, mode="teacher")
print("Evaluating Student on Test set ...")
student_test, s_probs, s_preds, _   = evaluate_model(student, test_loader, mode="student")

print("===== 📈 Test Metrics =====")
print(f"🧠 Teacher (XLM-R):      Acc={teacher_test['accuracy']:.4f} | "
      f"F1_macro={teacher_test['f1_macro']:.4f} | F1_weighted={teacher_test['f1_weighted']:.4f}")
print(f"🎓 Student (DistilBERT): Acc={student_test['accuracy']:.4f} | "
      f"F1_macro={student_test['f1_macro']:.4f} | F1_weighted={student_test['f1_weighted']:.4f}")

alignment = evaluate_alignment(teacher, student, test_loader)
print("\n===== 🔗 Alignment Metrics =====")
print(f"🔹 Logit Cosine Similarity : {alignment['logit_cosine']:.4f}")
print(f"🔹 Probability Correlation : {alignment['prob_corr']:.4f}")
print(f"🔹 Prediction Agreement    : {alignment['pred_alignment']:.4f}")

# ---------- save artifacts ----------
SAVE_DIR = WORK_DIR / "student_model_translit_attKD"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

torch.save(student.state_dict(), SAVE_DIR / "pytorch_model.bin")
student_tok.save_pretrained(SAVE_DIR)

with open(SAVE_DIR / "student_config.json", "w") as f:
    json.dump({
        "teacher_model": "xlm-roberta-base",
        "student_model": "distilbert-base-uncased",
        "num_labels": 2,
        "kd_temperature": 3.0,
        "alpha": 0.5,
        "gamma_hidden": 1.0,
        "gamma_attention": 1.0
    }, f, indent=2, ensure_ascii=False)

def _to_py(o):
    # ensure json-serializable (avoid numpy dtypes)
    if isinstance(o, dict): return {k: _to_py(v) for k, v in o.items()}
    if isinstance(o, (np.floating, np.float32, np.float64)): return float(o)
    return o

with open(WORK_DIR / "metrics_summary_translit_attKD.json", "w") as f:
    json.dump({
        "teacher_test": _to_py(teacher_test),
        "student_test": _to_py(student_test),
        "alignment": _to_py(alignment)
    }, f, indent=2, ensure_ascii=False)

print(f"\n✅ Saved student model + metrics → {WORK_DIR}")


[Teacher][Test]: {'accuracy': 0.9678238780694327, 'f1_macro': 0.9601959073041282, 'f1_weighted': 0.9678533866604009}
[Student][Test]: {'accuracy': 0.9331075359864521, 'f1_macro': 0.9162382020736879, 'f1_weighted': 0.9327574177354325}

🧩 Alignment Results (Test):
  🔹 Logit Cosine Similarity : 0.8767
  🔹 Probability Correlation : 0.8764
  🔹 Prediction Agreement    : 0.9382

✅ Saved model + metrics to /kaggle/working
